In [ ]:
import numpy as np  #linear algebra
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as  sns
import warnings #ignore warnings
warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/dataset/heart.csv')

In [ ]:
df.shape

(1025, 14)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1025 entries, 0 to 1024
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1025 non-null   int64  
 1   sex       1025 non-null   int64  
 2   cp        1025 non-null   int64  
 3   trestbps  1025 non-null   int64  
 4   chol      1025 non-null   int64  
 5   fbs       1025 non-null   int64  
 6   restecg   1025 non-null   int64  
 7   thalach   1025 non-null   int64  
 8   exang     1025 non-null   int64  
 9   oldpeak   1025 non-null   float64
 10  slope     1025 non-null   int64  
 11  ca        1025 non-null   int64  
 12  thal      1025 non-null   int64  
 13  target    1025 non-null   int64  
dtypes: float64(1), int64(13)
memory usage: 112.2 KB


In [ ]:
df.isnull().sum()

age         0
sex         0
cp          0
trestbps    0
chol        0
fbs         0
restecg     0
thalach     0
exang       0
oldpeak     0
slope       0
ca          0
thal        0
target      0
dtype: int64

In [ ]:
df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,52,1,0,125,212,0,1,168,0,1.0,2,2,3,0
1,53,1,0,140,203,1,0,155,1,3.1,0,0,3,0
2,70,1,0,145,174,0,1,125,1,2.6,0,0,3,0
3,61,1,0,148,203,0,1,161,0,0.0,2,1,3,0
4,62,0,0,138,294,1,1,106,0,1.9,1,3,2,0




*   age = age in years
*   sex = (1=male,0=female)
*   cp = chest pain type
*   trestbps = resting blood pressure (in mm Hg on admission to the hospital)
*   chol = cholestrol in Mg/dl
*   fbs = (fasting blood sugar = 120 mg/dl) (1 = true , 0 = false)
*   restecg = resting electrocardiographic results
*   thalach = maximum heart rate achieved
*   exang = excercise included angina (1 = yes , 0 = no)
*   oldpeak = ST depression induced by excercise relative to rest
*   slope = the slope of the peak excercise ST segment
*   ca = number of major vessels (0-3) coloured by flouroscopy
*   thal = 3=normal, 6=fixed detect, 7=reverseble defect
*   target = refers to the presence of heart disease in the patient (1=yes, 0=no)















In [ ]:
df.target.value_counts()

1    526
0    499
Name: target, dtype: int64

In [ ]:
df.isna().sum()

age         0
sex         0
cp          0
trestbps    0
chol        0
fbs         0
restecg     0
thalach     0
exang       0
oldpeak     0
slope       0
ca          0
thal        0
target      0
dtype: int64

In [ ]:
qualitative = []
quantitative = []
for feature in df.columns:
  if len(df[feature].unique()) <= 8:
    qualitative.append(feature)
  else:
    quantitative.append(feature)

In [ ]:
qualitative

['sex', 'cp', 'fbs', 'restecg', 'exang', 'slope', 'ca', 'thal', 'target']

In [ ]:
quantitative

['age', 'trestbps', 'chol', 'thalach', 'oldpeak']

In [ ]:
X = df.drop('target',1)
y = df['target']


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=42)


In [ ]:
X_train.shape

(820, 13)

In [ ]:
X_test.shape

(205, 13)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=4)  #max_depth=5, random_state=1
model.fit(X_train,y_train)
Y_pred  = model.predict(X_test)
score = model.score(X_train,y_train)
Score = model.score(X_test,y_test)
print('Testing score',Score)

Testing score 0.9707317073170731


Random forest with GridSearchCV


In [ ]:
#Hyperparameter tuning using GridSearchCV
from sklearn.model_selection import GridSearchCV
parameters = {'n_estimators':[100,200,300,400,500],
              'criterion':['gini', 'entropy', 'log_loss'],
              'max_features':['auto','sqrt','log2'],
              'max_depth':[4,5,6,7,8,9,10,20,30,40,50]
              }

In [ ]:
CV_model = GridSearchCV(RandomForestClassifier(),parameters,cv=5)
CV_model.fit(X_train,y_train)

GridSearchCV(cv=5, estimator=RandomForestClassifier(),
             param_grid={'criterion': ['gini', 'entropy', 'log_loss'],
                         'max_depth': [4, 5, 6, 7, 8, 9, 10, 20, 30, 40, 50],
                         'max_features': ['auto', 'sqrt', 'log2'],
                         'n_estimators': [100, 200, 300, 400, 500]})

In [ ]:
# Finding the best parameters for our model
final_model = CV_model.best_params_
final_model

{'criterion': 'gini',
 'max_depth': 50,
 'max_features': 'log2',
 'n_estimators': 100}

In [ ]:
final_model = RandomForestClassifier(criterion='gini',
                                     max_depth=50,
                                     max_features='log2',
                                     n_estimators=100)


In [ ]:
#fitting the training data  into the model
final_model.fit(X_train, y_train)

RandomForestClassifier(max_depth=50, max_features='log2')

In [ ]:
#Predicting the accuracy of the model
from sklearn.metrics import accuracy_score
pred1 = final_model.predict(X_test)
print('Model Accuracy:',accuracy_score(y_test,pred1))

Model Accuracy: 0.9853658536585366


NameError: ignored